In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.ion()

import qcodes as qc
from zhinst.qcodes import HDAWG

In [24]:
import time

In [2]:
hd = HDAWG("hdawg1", "dev8030")

Successfully connected to data server at localhost8004 api version: 6
Successfully connected to device DEV8030 on interface 1GBE
Connected to: Zurich Instruments HDAWG (serial:dev8030, firmware:65991) in 1.68s


## HDAWG

In [3]:
hd.print_readable_snapshot()

hdawg1:
	parameter value
--------------------------------------------------------------------------------
IDN       :	{'vendor': 'Zurich Instruments', 'model': 'HDAWG', 'serial': 'dev...
clockbase :	None (Hz)
hdawg1_stats:
	parameter                                        value
--------------------------------------------------------------------------------
no parameters
hdawg1_stats_physical:
	parameter      value
--------------------------------------------------------------------------------
overtemperature :	None 
temperatures0   :	None (°C)
temperatures1   :	None (°C)
temperatures10  :	None (°C)
temperatures11  :	None (°C)
temperatures12  :	None (°C)
temperatures13  :	None (°C)
temperatures14  :	None (°C)
temperatures15  :	None (°C)
temperatures16  :	None (°C)
temperatures17  :	None (°C)
temperatures18  :	None (°C)
temperatures19  :	None (°C)
temperatures2   :	None (°C)
temperatures20  :	None (°C)
temperatures21  :	None (°C)
temperatures22  :	None (°C)
temperatures23  :	None (°C)


In [5]:
hd.awgs

ChannelList(<HDAWG: hdawg1>, AWG, (<AWG: hdawg1_awg-0 of HDAWG: hdawg1>, <AWG: hdawg1_awg-1 of HDAWG: hdawg1>, <AWG: hdawg1_awg-2 of HDAWG: hdawg1>, <AWG: hdawg1_awg-3 of HDAWG: hdawg1>))

In [9]:
awg = hd.awgs[0]

In [10]:
awg.parameters

{'outputs': <qcodes.instrument.parameter.Parameter: outputs at 2242324440136>,
 'output1': <qcodes.instrument.parameter.Parameter: output1 at 2242324440968>,
 'output2': <qcodes.instrument.parameter.Parameter: output2 at 2242324441800>,
 'gain1': <qcodes.instrument.parameter.Parameter: gain1 at 2242324442440>,
 'gain2': <qcodes.instrument.parameter.Parameter: gain2 at 2242324443080>,
 'modulation_phase_shift': <qcodes.instrument.parameter.Parameter: modulation_phase_shift at 2242324460168>,
 'modulation_freq': <qcodes.instrument.parameter.Parameter: modulation_freq at 2242324460808>}

In [42]:
awg.set_sequence_params(
    sequence_type="Simple",
    repetitions=100,
    period=20e-6,
    trigger_mode="Send Trigger",
    alignment="End with Trigger",
)

In [ ]:
awg.sequence_params()

In [43]:
N = 50

awg.reset_queue()
for i in range(N):
    w = i/N * np.ones(1000)
    awg.queue_waveform(w, w)

Current length of queue: 1
Current length of queue: 2
Current length of queue: 3
Current length of queue: 4
Current length of queue: 5


In [44]:
awg.compile_and_upload_waveforms()

Compilation successful
hdawg1-0: Sequencer status: ELF file uploaded
Upload of 5 waveforms took 0.048045 s


In [45]:
awg.outputs(("on", "off"))

In [48]:
awg.enable_iq_modulation()
awg.modulation_phase_shift(-90)

In [50]:
freqs = np.linspace(10e6, 100e6, 91)

for f in freqs:
    awg.modulation_freq(f)
    awg.run()
    awg.wait_done()